In [92]:
import pandas as pd
import numpy as np

In [93]:
safety_car = pd.read_csv('../../data/safety_car.csv')
races = pd.read_csv('../clean_data/races_clean.csv')

# Análisis del dataframe

---

Procedemos a transformar nuestro dataframe según nuestros objetivos, eliminando ciertas columnas y modificando ciertos parámetros

In [94]:
safety_car.shape

(301, 6)

In [95]:
safety_car.head()

,Grand Prix,Cause,Deployed (lap),Called in (lap),Number of full laps,Year
0,São Paulo Grand Prix,Accident (Ricciardo and Magnussen),1,6,5,2022
1,Miami Grand Prix,Accident,42,46,3,2022
2,Emilia Romagna Grand Prix,Accident (Sainz),1,4,2,2022
3,Emilia Romagna Sprint,Accident (Zhou),1,4,2,2022
4,Australian Grand Prix,Accident (Vettel),23,26,2,2022


In [96]:
safety_car.rename(columns={'Grand Prix': 'grand_prix', 'Cause': 'cause', 'Deployed (lap)': 'deployed_lap', 'Called in (lap)': 'called_in_lap', 
                           'Number of full laps': 'number_full_laps', 'Year': 'year'}, inplace=True)
safety_car['grand_prix'] = safety_car['grand_prix'].str.replace('Emilia-Romagna Grand Prix', 'Emilia Romagna Grand Prix')       # modificamos estos valores
safety_car['grand_prix'] = safety_car['grand_prix'].str.replace('Emilia Romagna Sprint', 'Emilia Romagna Grand Prix')           # para que hagan match con el
safety_car['safetyId'] = range(1, len(safety_car) + 1)      # creamos la columna 'safetyId'                                       # dataframe de 'races'

safety_car.head()

,grand_prix,cause,deployed_lap,called_in_lap,number_full_laps,year,safetyId
0,São Paulo Grand Prix,Accident (Ricciardo and Magnussen),1,6,5,2022,1
1,Miami Grand Prix,Accident,42,46,3,2022,2
2,Emilia Romagna Grand Prix,Accident (Sainz),1,4,2,2022,3
3,Emilia Romagna Grand Prix,Accident (Zhou),1,4,2,2022,4
4,Australian Grand Prix,Accident (Vettel),23,26,2,2022,5


### Asociamos los Grand Prix

---

Procedemos a anclar cada Grand Prix del dataframe de safety cars a un ID que se encuentra en el dataframe de races, para así poder manejarnos entre ambos dataframes

In [97]:
races = races.drop(columns='Unnamed: 0', axis=1)

races.head()

,raceId,year,round,circuitId,name,date
0,1,2009,1,1,Australian Grand Prix,2009-03-29
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05
2,3,2009,3,17,Chinese Grand Prix,2009-04-19
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26
4,5,2009,5,4,Spanish Grand Prix,2009-05-10


In [98]:
def get_raceId(row):
    race = races[(races['name'] == row['grand_prix']) & (races['year'] == row['year'])]['raceId']       # obtener los 'raceId' que hagan match con el gran premio y el año
    return race.values[0] if not race.empty else None

safety_car['raceId'] = safety_car.apply(get_raceId, axis=1)     # aplicamos la función

safety_car.head()

,grand_prix,cause,deployed_lap,called_in_lap,number_full_laps,year,safetyId,raceId
0,São Paulo Grand Prix,Accident (Ricciardo and Magnussen),1,6,5,2022,1,NaN
1,Miami Grand Prix,Accident,42,46,3,2022,2,1078.0
2,Emilia Romagna Grand Prix,Accident (Sainz),1,4,2,2022,3,1077.0
3,Emilia Romagna Grand Prix,Accident (Zhou),1,4,2,2022,4,1077.0
4,Australian Grand Prix,Accident (Vettel),23,26,2,2022,5,1076.0


In [99]:
safety_car.at[0, 'raceId'] = 1095.0     # cambiamos manualmente el valor en la primera fila de la columna 'raceId' porque hay una diferencia en los nombres de ambos dataframes
safety_car['raceId'] = safety_car['raceId'].astype(int)     # cambiamos el tipo de dato al no haber más nulos

safety_car.head()

,grand_prix,cause,deployed_lap,called_in_lap,number_full_laps,year,safetyId,raceId
0,São Paulo Grand Prix,Accident (Ricciardo and Magnussen),1,6,5,2022,1,1095
1,Miami Grand Prix,Accident,42,46,3,2022,2,1078
2,Emilia Romagna Grand Prix,Accident (Sainz),1,4,2,2022,3,1077
3,Emilia Romagna Grand Prix,Accident (Zhou),1,4,2,2022,4,1077
4,Australian Grand Prix,Accident (Vettel),23,26,2,2022,5,1076


In [100]:
safety_car = safety_car[['safetyId', 'raceId', 'grand_prix', 'cause', 'deployed_lap', 'called_in_lap', 'number_full_laps', 'year']]

safety_car.head()       # reordenamos las columnas a nuestro gusto

,safetyId,raceId,grand_prix,cause,deployed_lap,called_in_lap,number_full_laps,year
0,1,1095,São Paulo Grand Prix,Accident (Ricciardo and Magnussen),1,6,5,2022
1,2,1078,Miami Grand Prix,Accident,42,46,3,2022
2,3,1077,Emilia Romagna Grand Prix,Accident (Sainz),1,4,2,2022
3,4,1077,Emilia Romagna Grand Prix,Accident (Zhou),1,4,2,2022
4,5,1076,Australian Grand Prix,Accident (Vettel),23,26,2,2022


In [101]:
safety_car['deployed_lap'].unique()     # no podemos cambiar el tipo de dato a int

array(['1', '42', '23', '3', '16', '46', '2', '32', '8', '31', '47',
       '1 (Race started behind SC)', '26', '6', '10', '53', '20', '11',
       '13', '55', '51', '25', '44', '7', '62', '45', '57', '29', '41',
       '36', '50', '54', '66', '33', '40', '28', '39', '4', '52', '60',
       '12', '17', '38', '30', '18', '27', '21 (Race resumed behind SC)',
       '48', '64', '15', '43', '37', '68', '3 (Race resumed behind SC)',
       '47 (Resumed behind SC)', '63', '24', '70', '9',
       '10 (Race resumed behind SC)', '35', '26 (Race resumed behind SC)',
       '56', '75', '22', '4 (Race resumed behind SC)', '19', '21', '14',
       '61', '5'], dtype=object)

In [102]:
safety_car['called_in_lap'].unique()        # no podemos cambiar el tipo de dato a int

array(['6', '46', '4', '26', '20', '50', '4 (After red flag)', '3',
       '34 (After red flag)', '10', '35', '49 (After red flag)',
       'NA (Red flag)', '30', '9', '14 (After red flag)', '57',
       '3 (After red flag)', '5', '23', '14', '18', '60', '54',
       '28 (After red flag)', '46 (After red flag)', '9 (After red flag)',
       '8', '13', 'NA (Race finished behind SC)', '68', '49', '59', '52',
       '33', '45', '40', '47', '51', '32', '69', '37', '31', '41', '7',
       '16', '66', '19', '23 (After red flag)', '19 (After red flag)',
       '2', '20 (Red Flag)', '32 (After red flag)', '55', '48', '70',
       '70 (finished under the SC)', '2 (After red flag)', '28', '38',
       '67', '21', '29', '36', '42', '15', '10 (After red flag)',
       '26 (After red flag)', '39', '34', '61', '17', '24', '56', '22',
       '27', '11', '53', '7 (After red flag)', '64', '25', '12'],
      dtype=object)

In [103]:
safety_car.isna().sum()

safetyId            0
raceId              0
grand_prix          0
cause               0
deployed_lap        0
called_in_lap       0
number_full_laps    0
year                0
dtype: int64

In [104]:
safety_car.dtypes

safetyId             int64
raceId               int32
grand_prix          object
cause               object
deployed_lap        object
called_in_lap       object
number_full_laps     int64
year                 int64
dtype: object

In [105]:
safety_clean = safety_car.to_csv('../clean_data/safety_clean.csv')